# Interactive Brokers TWS API Demo

This notebook demonstrates the major capabilities of the IB TWS API using the `ib_insync` library — a high-level, Pythonic wrapper around the official Interactive Brokers API.

### Prerequisites
1. TWS (Trader Workstation) or IB Gateway must be running
2. API connections must be enabled in TWS:  
   `Edit → Global Configuration → API → Settings`  
   - Check "Enable ActiveX and Socket Clients"  
   - Socket port: 7497 (paper trading) or 7496 (live trading)
3. Install dependencies: `pip install -r requirements.txt`

**IMPORTANT:** This demo uses a PAPER TRADING port (7497) by default. Placing orders on a live account will use REAL MONEY.

---
## Setup & Imports

In [ ]:
import time
from datetime import datetime, timedelta

from ib_insync import (
    IB,
    Stock,
    Forex,
    Future,
    Option,
    Contract,
    MarketOrder,
    LimitOrder,
    StopOrder,
    ScannerSubscription,
    util,
)

# Enable ib_insync's Jupyter integration
util.startLoop()

print("Imports loaded successfully!")

In [ ]:
# Configuration
TWS_HOST = "127.0.0.1"   # TWS/Gateway runs locally
TWS_PORT = 7497           # 7497 = paper trading, 7496 = live trading
CLIENT_ID = 1             # Unique ID for this API client

# Create the IB instance (the main API client)
ib = IB()

print(f"Host: {TWS_HOST}")
print(f"Port: {TWS_PORT} ({'Paper Trading' if TWS_PORT == 7497 else 'Live Trading'})")
print(f"Client ID: {CLIENT_ID}")

---
## Section 1: Connection

The `IB` class is the main entry point. It manages:
- The TCP socket connection to TWS
- Request/response message handling
- An internal event loop for async operations

In [ ]:
ib.connect(TWS_HOST, TWS_PORT, clientId=CLIENT_ID)

print(f"Connected:        {ib.isConnected()}")
print(f"TWS Server Time:  {ib.reqCurrentTime()}")
print(f"Managed Accounts: {ib.managedAccounts()}")

---
## Section 2: Defining Contracts

A **Contract** tells IB which instrument you want to trade or get data for. Every request (market data, orders, etc.) requires a contract object.

Key fields:
- `symbol` — Ticker symbol (e.g. 'AAPL')
- `secType` — Security type: 'STK', 'FX', 'FUT', 'OPT', etc.
- `exchange` — 'SMART' for best routing, or specific like 'NASDAQ'
- `currency` — Currency the instrument is denominated in

In [ ]:
# US Stock — SMART exchange = IB's Smart Order Routing (best price across exchanges)
apple = Stock("AAPL", "SMART", "USD")
print(f"Stock:   {apple}")

In [ ]:
# Forex pair — uses IDEALPRO exchange
eurusd = Forex("EURUSD")
print(f"Forex:   {eurusd}")

In [ ]:
# Futures — need lastTradeDateOrContractMonth to identify the specific contract
es_future = Future("ES", "202503", "CME")  # E-mini S&P 500, March 2025
print(f"Future:  {es_future}")

In [ ]:
# Options — need: expiry, strike, right ('C' = call, 'P' = put)
aapl_call = Option("AAPL", "20250321", 200, "C", "SMART")
print(f"Option:  {aapl_call}")

In [ ]:
# Generic contract by conId — every instrument has a unique contract ID
contract_by_id = Contract(conId=265598)  # This is AAPL
print(f"By ID:   {contract_by_id}")

---
## Section 3: Contract Details & Search

`reqContractDetails()` returns full information about a contract:
- Full name, valid exchanges, trading hours
- Minimum tick size, multiplier
- Industry, category, subcategory
- Supported order types

This is also how you **validate** a contract before using it.

In [ ]:
# Qualify the contract — fills in missing fields (conId, exchange, etc.)
# and verifies the contract exists at IB
ib.qualifyContracts(apple)

print(f"Qualified contract: {apple}")
print(f"conId:              {apple.conId}")

In [ ]:
# Get full contract details
details_list = ib.reqContractDetails(apple)

if details_list:
    details = details_list[0]
    print(f"Long Name:        {details.longName}")
    print(f"Industry:         {details.industry}")
    print(f"Category:         {details.category}")
    print(f"Min Tick:         {details.minTick}")
    print(f"Valid Exchanges:  {details.validExchanges}")

In [ ]:
# Search for contracts by partial name
matches = ib.reqMatchingSymbols("Tesla")

if matches:
    print(f"Search for 'Tesla' returned {len(matches)} results:")
    for m in matches[:3]:
        print(f"  {m.contract.symbol} — {m.contract.secType} — {m.contract.primaryExchange}")

---
## Section 4: Live Market Data (Streaming)

`reqMktData()` streams real-time market data (bid, ask, last, volume, etc.).

Generic tick types for additional data:
- 100: Option volume
- 101: Option open interest
- 104: Historical volatility
- 106: Implied volatility
- 221: Mark price
- 233: RTVolume (every trade)
- 236: Shortable shares
- 258: Fundamental ratios

**Note:** Live data requires subscriptions. Use delayed data (type 3) if you don't have them.

In [ ]:
# Request delayed data (comment out if you have live data subscriptions)
ib.reqMarketDataType(3)  # 1=Live, 2=Frozen, 3=Delayed, 4=Delayed-Frozen

# Start streaming market data
ticker = ib.reqMktData(apple, genericTickList="", snapshot=False)

# Give it a moment to receive the first ticks
ib.sleep(2)

print(f"Symbol:     {apple.symbol}")
print(f"Bid:        {ticker.bid}")
print(f"Ask:        {ticker.ask}")
print(f"Last:       {ticker.last}")
print(f"Volume:     {ticker.volume}")
print(f"High:       {ticker.high}")
print(f"Low:        {ticker.low}")
print(f"Close:      {ticker.close}")

In [ ]:
# Cancel the streaming data when done to free up the data line
# IB limits simultaneous streaming requests (typically 100)
ib.cancelMktData(apple)
print("Streaming market data cancelled.")

---
## Section 5: Market Data Snapshot

A snapshot requests a **one-time** quote instead of continuous streaming. Useful when you just need the current price without ongoing updates.

In [ ]:
ib.reqMarketDataType(3)  # Delayed data

# snapshot=True gets a single quote and auto-cancels
ticker = ib.reqMktData(eurusd, snapshot=True)
ib.sleep(2)

print(f"{eurusd.symbol}{eurusd.currency} snapshot:")
print(f"  Bid:  {ticker.bid}")
print(f"  Ask:  {ticker.ask}")
print(f"  Last: {ticker.last}")

---
## Section 6: Historical Data

`reqHistoricalData()` returns OHLCV bars (candles) for any timeframe.

Parameters:
- `durationStr` — How far back: '1 D', '1 W', '1 M', '1 Y', '30 D', etc.
- `barSizeSetting` — Bar size: '1 min', '5 mins', '15 mins', '1 hour', '1 day', '1 week', '1 month'
- `whatToShow` — 'TRADES', 'MIDPOINT', 'BID', 'ASK', 'ADJUSTED_LAST', etc.
- `useRTH` — 1 = Regular Trading Hours only, 0 = include extended hours

**Pacing limits:** Max 60 requests in 10 minutes.

In [ ]:
# Get 30 days of daily bars
bars = ib.reqHistoricalData(
    apple,
    endDateTime="",          # '' means "up to now"
    durationStr="30 D",      # 30 days of data
    barSizeSetting="1 day",  # Daily candles
    whatToShow="TRADES",     # Trade prices
    useRTH=True,             # Regular Trading Hours only
    formatDate=1,            # 1 = yyyyMMdd format
)

if bars:
    print(f"Received {len(bars)} daily bars for {apple.symbol}")
    print(f"{'Date':<12} {'Open':>8} {'High':>8} {'Low':>8} {'Close':>8} {'Volume':>10}")
    print(f"{'-'*12} {'-'*8} {'-'*8} {'-'*8} {'-'*8} {'-'*10}")

    for bar in bars[-5:]:
        print(f"{str(bar.date):<12} {bar.open:>8.2f} {bar.high:>8.2f} "
              f"{bar.low:>8.2f} {bar.close:>8.2f} {bar.volume:>10}")
else:
    print("No historical data returned (check market data subscriptions)")

In [ ]:
# Convert to a pandas DataFrame for further analysis
if bars:
    df = util.df(bars)
    df.tail()

---
## Section 7: Account & Portfolio

Account and portfolio information:
- `accountSummary()` — Key financial metrics (net liquidation, buying power, etc.)
- `portfolio()` — Current open positions with market value and P&L
- `positions()` — Simplified position list across all accounts

These update automatically once requested.

In [ ]:
# Account Summary
summary = ib.accountSummary()

print("Account Summary (selected fields):")
for item in summary:
    if item.tag in ("NetLiquidation", "TotalCashValue", "BuyingPower", "MaintMarginReq"):
        print(f"  {item.tag:<25} {item.value:>15} {item.currency}")

In [ ]:
# Portfolio (open positions)
portfolio = ib.portfolio()

print(f"Open Positions: {len(portfolio)}")
for pos in portfolio[:5]:
    print(f"  {pos.contract.symbol:<8} "
          f"Qty: {pos.position:>8} "
          f"Avg Cost: {pos.averageCost:>10.2f} "
          f"Market Value: {pos.marketValue:>12.2f} "
          f"Unrealized P&L: {pos.unrealizedPNL:>10.2f}")

In [ ]:
# Positions (simpler, across all accounts)
positions = ib.positions()
print(f"Total positions across all accounts: {len(positions)}")
for pos in positions:
    print(f"  {pos.contract.symbol}: {pos.position} shares")

---
## Section 8: Placing Orders

Order types supported:
- **MarketOrder** — Execute immediately at market price
- **LimitOrder** — Execute at specified price or better
- **StopOrder** — Becomes market order when stop price is hit
- **BracketOrder** — Entry + take-profit + stop-loss (3 linked orders)

**CAUTION:** On a live account, these orders will execute with REAL MONEY.

In [ ]:
# Market Order — executes immediately at the best available price
market_order = MarketOrder("BUY", 1)
print(f"Market Order: {market_order}")
# To submit: trade = ib.placeOrder(apple, market_order)

In [ ]:
# Limit Order — only executes at the specified price or better
limit_order = LimitOrder("BUY", 1, limitPrice=150.00)
print(f"Limit Order: {limit_order}")
# To submit: trade = ib.placeOrder(apple, limit_order)

In [ ]:
# Stop Order — becomes a market order when the stop price is reached
stop_order = StopOrder("SELL", 1, stopPrice=140.00)
print(f"Stop Order: {stop_order}")
# To submit: trade = ib.placeOrder(apple, stop_order)

In [ ]:
# Bracket Order — creates 3 linked orders:
#   1. Entry order (limit buy)
#   2. Take-profit order (limit sell at higher price)
#   3. Stop-loss order (stop sell at lower price)
# When one exit order fills, the other is automatically cancelled (OCA group).

bracket = ib.bracketOrder(
    action="BUY",
    quantity=1,
    limitPrice=150.00,      # Entry price
    takeProfitPrice=170.00, # Take profit at $170
    stopLossPrice=140.00,   # Stop loss at $140
)

labels = ["Entry", "Take Profit", "Stop Loss"]
for i, order in enumerate(bracket):
    print(f"  {labels[i]}: {order.orderType} — "
          f"{'Limit: ' + str(order.lmtPrice) if hasattr(order, 'lmtPrice') and order.lmtPrice else ''}"
          f"{'Stop: ' + str(order.auxPrice) if hasattr(order, 'auxPrice') and order.auxPrice else ''}")

print("\n(Orders are created but NOT submitted — uncomment placeOrder calls to place real orders)")

---
## Section 9: Order Management (Modify & Cancel)

After placing an order, you can:
- **Modify:** `ib.placeOrder(contract, modified_order)` (resubmit with same orderId)
- **Cancel:** `ib.cancelOrder(order)`
- **View all:** `ib.openTrades()` / `ib.orders()`

In [ ]:
# View all open orders
open_trades = ib.openTrades()
print(f"Open trades: {len(open_trades)}")

for trade in open_trades:
    print(f"  {trade.contract.symbol} {trade.order.action} "
          f"{trade.order.totalQuantity} @ {trade.order.orderType} "
          f"— Status: {trade.orderStatus.status}")

# To modify an order:
#   trade.order.lmtPrice = 155.00
#   ib.placeOrder(trade.contract, trade.order)
#
# To cancel an order:
#   ib.cancelOrder(trade.order)
#
# To cancel ALL open orders:
#   ib.reqGlobalCancel()

---
## Section 10: News

News API capabilities:
- `reqNewsProviders()` — List available news sources
- `reqHistoricalNews()` — Headlines for a specific contract
- `reqNewsArticle()` — Full text of a specific article

**Note:** News requires appropriate market data subscriptions.

In [ ]:
# List available news providers
providers = ib.reqNewsProviders()

print(f"News providers available: {len(providers)}")
for p in providers:
    print(f"  {p.code}: {p.name}")

In [ ]:
# Request recent headlines for AAPL
if apple.conId and providers:
    headlines = ib.reqHistoricalNews(
        conId=apple.conId,
        providerCodes="+".join(p.code for p in providers[:3]),
        startDateTime="",
        endDateTime="",
        totalResults=5,
    )
    print(f"Recent headlines for {apple.symbol}:")
    for h in headlines:
        print(f"  [{h.time}] {h.headline}")
else:
    print("Qualify the apple contract first or no news providers available")

---
## Section 11: Option Chains

Option chain retrieval:
1. `reqSecDefOptParams()` — Get available expirations and strikes
2. Create Option contracts for specific strikes/expirations
3. Request market data or place orders on those options

In [ ]:
chains = ib.reqSecDefOptParams(
    apple.symbol, "",
    apple.secType, apple.conId
)

if chains:
    chain = chains[0]
    print(f"Exchange:   {chain.exchange}")
    print(f"Multiplier: {chain.multiplier}")

    expirations = sorted(chain.expirations)[:5]
    print(f"Next 5 Expirations: {expirations}")

    strikes = sorted(chain.strikes)
    mid_idx = len(strikes) // 2
    nearby_strikes = strikes[max(0, mid_idx - 3):mid_idx + 3]
    print(f"Sample Strikes: {nearby_strikes}")
else:
    print("No option chain data returned")

In [ ]:
# Create and qualify a sample option contract
if chains:
    chain = chains[0]
    expirations = sorted(chain.expirations)[:5]
    strikes = sorted(chain.strikes)
    mid_idx = len(strikes) // 2
    nearby_strikes = strikes[max(0, mid_idx - 3):mid_idx + 3]

    if expirations and nearby_strikes:
        opt = Option(
            apple.symbol,
            expirations[0],
            nearby_strikes[0],
            "C",  # Call
            chain.exchange,
        )
        ib.qualifyContracts(opt)
        print(f"Sample Option Contract: {opt}")

---
## Section 12: Market Scanners

Market scanners find instruments matching specific criteria.

Popular scan codes:
- `TOP_PERC_GAIN` — Top percentage gainers
- `TOP_PERC_LOSE` — Top percentage losers
- `MOST_ACTIVE` — Highest volume
- `HOT_BY_VOLUME` — Unusual volume
- `HIGH_OPT_IMP_VOLAT` — High implied volatility
- `TOP_TRADE_COUNT` — Most trades

In [ ]:
scanner = ScannerSubscription(
    instrument="STK",
    locationCode="STK.US.MAJOR",
    scanCode="TOP_PERC_GAIN",
    numberOfRows=10,
)
scanner.abovePrice = 5.0
scanner.belowPrice = 500.0
scanner.aboveVolume = 100000

results = ib.reqScannerData(scanner)

print(f"Top {len(results)} Gainers (US Stocks, $5-$500, vol > 100k):")
print(f"{'Rank':<6} {'Symbol':<8} {'SecType':<8}")
print(f"{'-'*6} {'-'*8} {'-'*8}")

for item in results:
    print(f"{item.rank:<6} {item.contractDetails.contract.symbol:<8} "
          f"{item.contractDetails.contract.secType:<8}")

---
## Section 13: Real-Time Bars (5-Second Candles)

`reqRealTimeBars()` streams live 5-second candles with open, high, low, close, volume, and VWAP.

Only 5-second bars are supported (no other intervals). `whatToShow` can be: TRADES, BID, ASK, or MIDPOINT.

In [ ]:
bars_rt = ib.reqRealTimeBars(apple, barSize=5, whatToShow="TRADES", useRTH=True)

print(f"Streaming 5-second bars for {apple.symbol} (15 seconds)...")

for i in range(3):
    ib.sleep(5)
    if bars_rt:
        bar = bars_rt[-1]
        print(f"  {bar.time} — O:{bar.open:.2f} H:{bar.high:.2f} "
              f"L:{bar.low:.2f} C:{bar.close:.2f} V:{bar.volume}")

ib.cancelRealTimeBars(bars_rt)
print("Streaming stopped.")

---
## Section 14: Profit & Loss

Real-time P&L tracking:
- `reqPnL()` — Total account P&L (daily, unrealized, realized)
- `reqPnLSingle()` — P&L for a specific position (by conId)

In [ ]:
account = ib.managedAccounts()[0]

pnl = ib.reqPnL(account)
ib.sleep(1)

print(f"Account: {account}")
print(f"Daily P&L:      {pnl.dailyPnL}")
print(f"Unrealized P&L: {pnl.unrealizedPnL}")
print(f"Realized P&L:   {pnl.realizedPnL}")

ib.cancelPnL(pnl)

---
## Section 15: What-If Orders (Order Preview)

"What-If" orders preview the impact of an order **without placing it**:
- Estimated commission
- Margin impact (initial and maintenance)
- Equity with loan value

Invaluable for checking margin requirements before placing orders.

In [ ]:
# What-If order preview — does NOT place a real order
order_preview = MarketOrder("BUY", 100)
order_preview.whatIf = True

preview_trade = ib.placeOrder(apple, order_preview)
ib.sleep(1)

status = preview_trade.orderStatus
print(f"What-If: BUY 100 {apple.symbol}")
print(f"Commission:     {status.commission}")
print(f"Init Margin:    {status.initMarginChange}")
print(f"Maint Margin:   {status.maintMarginChange}")
print(f"Equity w/ Loan: {status.equityWithLoanChange}")

---
## Section 16: Event Callbacks

The IB API is event-driven. Available events:
- `pendingTickersEvent` — When tickers have new data
- `orderStatusEvent` — When an order status changes
- `newOrderEvent` — When a new order is detected
- `execDetailsEvent` — When an order gets filled
- `errorEvent` — When IB sends an error/warning
- `connectedEvent` / `disconnectedEvent` — Connection state changes
- `barUpdateEvent` — When real-time bars update
- `positionEvent` — When positions change
- `accountValueEvent` — When account values change

In [ ]:
# Define callback functions
def on_pending_tickers(tickers):
    for t in tickers:
        print(f"  [Tick] {t.contract.symbol}: bid={t.bid}, ask={t.ask}, last={t.last}")

def on_error(reqId, errorCode, errorString, contract):
    print(f"  [Error] reqId={reqId}, code={errorCode}: {errorString}")

# Register callbacks
ib.pendingTickersEvent += on_pending_tickers
ib.errorEvent += on_error

# Subscribe to market data to trigger the ticker callback
ib.reqMarketDataType(3)
ticker = ib.reqMktData(apple)

print(f"Listening for events on {apple.symbol} for 5 seconds...")
ib.sleep(5)

# Clean up
ib.pendingTickersEvent -= on_pending_tickers
ib.errorEvent -= on_error
ib.cancelMktData(apple)
print("Event listeners removed.")

---
## Disconnect

Always disconnect cleanly when done.

In [ ]:
if ib.isConnected():
    ib.disconnect()
    print("Disconnected from TWS.")
else:
    print("Already disconnected.")